# NFL Big Data Bowl - Feature Engineering v1
-------------------
TheNerdyCat <br>
Created: 2019/10/27 <br>
Deadline: 2019/11/27

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
#import matplotlib.patches as patches
import seaborn as sns

import kaggle
import math
import datetime
import random
from string import punctuation
import re

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GroupKFold, train_test_split

#from sklearn.model_selection import GridSearchCV
#import lightgbm as lgb
import keras
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
import keras.backend as K
import tensorflow as tf
import gc
#import optuna

from sklearn.metrics import mean_absolute_error

import tqdm
import warnings
warnings.filterwarnings('ignore')

sns.set_style('darkgrid')
mpl.rcParams['figure.figsize'] = [15,10]
pd.options.display.max_columns = 100

In [ ]:
df = pd.read_csv("../input/nfl-big-data-bowl-2020/train.csv", low_memory=False)
df.shape

In [ ]:
def preprocess(df):

    Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 
            'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 
            'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 
            'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 
            'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 
    
    # from https://www.kaggle.com/bgmello/neural-networks-feature-engineering-for-the-win
    map_abbr = {'ARI': 'ARZ', 'BAL': 'BLT', 'CLE': 'CLV', 'HOU': 'HST'}
    for abb in df['PossessionTeam'].unique():
        map_abbr[abb] = abb
    
    def clean_position(pos):
        if pos == 'SAF':
            return 'DB'
        if pos == 'S':
            return 'DB'
        elif pos == 'OG':
            return 'G'
        elif pos == "OT":
            return 'T'
        else:
            return pos
    
    def clean_offenceformation(of):
        if of == "SHOTGUN":
            return 9
        elif of == "SINGLEBACK":
            return 8
        elif of == "JUMBO":
            return 6
        elif of == "PISTOL":
            return 5
        elif of == "I_FORM":
            return 4
        elif of == "ACE":
            return 3
        elif of ==  "WILDCAT":
            return 2
        elif of == "EMPTY":
            return 1
        else: 
            return 7
    
    def create_generalposition(pos):
        if pos == 'SS' or pos == 'FS' or pos == 'CB' or pos == 'DB':
            return 'DB'
        elif pos == 'DE' or pos == 'DT' or pos == 'DL':
            return 'DL'
        elif pos == 'ILB' or pos == 'OLB' or pos == 'MLB' or pos == 'LB':
            return 'LB'
        elif pos == 'WR':
            return 'WR'
        elif pos == 'TE':
            return 'TE'
        elif pos == 'T' or pos == 'G' or pos == 'C' or pos == 'NT' or pos == 'OL':
            return 'OL'
        elif pos == 'QB' or pos == 'RB' or pos == 'FB' or pos == 'HB' or pos == 'TB' or pos == 'WB':
            return 'OB'
        else:
            return 'Other'
    
    def utc2sec(x):
        return int(x.split("-")[2].split(":")[2].split(".")[0])
    def timesnap2date(x):
        day = x.split("T")[0]
        return day
    def timesnap2day(x):
        days = x.split("-")
        return 365 * int(days[0]) + 30 * int(days[1]) + int(days[2][:2])
    def gameclock2secs(x):
        clock = x.split(":")
        return (60 * int(clock[0])) + int(clock[1])        
        
    def group_stadium_types(stadium):
        outdoor = [
            'Outdoor', 'Outdoors', 'Cloudy', 'Heinz Field', 'Outdor', 'Ourdoor', 
            'Outside', 'Outddors','Outdoor Retr Roof-Open', 'Oudoor', 'Bowl'
            ]
        indoor_closed = [
            'Indoors', 'Indoor', 'Indoor, Roof Closed', 'Indoor, Roof Closed',
            'Retractable Roof', 'Retr. Roof-Closed', 'Retr. Roof - Closed', 'Retr. Roof Closed',
        ]
        indoor_open   = ['Indoor, Open Roof', 'Open', 'Retr. Roof-Open', 'Retr. Roof - Open']
        dome_closed   = ['Dome', 'Domed, closed', 'Closed Dome', 'Domed', 'Dome, closed']
        dome_open     = ['Domed, Open', 'Domed, open']
        if stadium in outdoor:
            return 0 #'outdoor'
        elif stadium in indoor_closed:
            return 3 # 'indoor closed'
        elif stadium in indoor_open:
            return 2 #'indoor open'
        elif stadium in dome_closed:
            return 4 #'dome closed'
        elif stadium in dome_open:
            return 1 #'dome open'
        else:
            return 5 #'unknown'
        
    def map_weather(txt):
        ans = 1
        if pd.isna(txt):
            return 0
        if 'partly' in txt:
            ans*=0.5
        if 'climate controlled' in txt or 'indoor' in txt:
            return ans*3
        if 'sunny' in txt or 'sun' in txt:
            return ans*2
        if 'clear' in txt:
            return ans
        if 'cloudy' in txt:
            return -ans
        if 'rain' in txt or 'rainy' in txt:
            return -2*ans
        if 'snow' in txt:
            return -3*ans
        return 0       
        
    def clean_wind_speed(windspeed):
        """
        This is not a very robust function,
        but it should do the job for this dataset.
        """
        ws = str(windspeed)
        # if it's already a number just return an int value
        if ws.isdigit():
            return int(ws)
        # if it's a range, take their mean
        if '-' in ws:
            return (int(ws.split('-')[0]) + int(ws.split('-')[1]))/2
        # if there's a space between the number and mph
        if ws.split(' ')[0].isdigit():
            return int(ws.split(' ')[0])
        # if it looks like '10MPH' or '12mph' just take the first part
        if 'mph' in ws.lower():
            return int(ws.lower().split('mph')[0])
        else:
            return 0   
            
    def clean_WindDirection(txt):
        if pd.isna(txt):
            return np.nan
        txt = txt.lower()
        txt = ''.join([c for c in txt if c not in punctuation])
        txt = txt.replace('from', '')
        txt = txt.replace(' ', '')
        txt = txt.replace('north', 'n')
        txt = txt.replace('south', 's')
        txt = txt.replace('west', 'w')
        txt = txt.replace('east', 'e')
        return txt         
        
    def transform_WindDirection(txt):
        if pd.isna(txt):
            return np.nan
        
        if txt=='n':
            return 0
        if txt=='nne' or txt=='nen':
            return 1/8
        if txt=='ne':
            return 2/8
        if txt=='ene' or txt=='nee':
            return 3/8
        if txt=='e':
            return 4/8
        if txt=='ese' or txt=='see':
            return 5/8
        if txt=='se':
            return 6/8
        if txt=='ses' or txt=='sse':
            return 7/8
        if txt=='s':
            return 8/8
        if txt=='ssw' or txt=='sws':
            return 9/8
        if txt=='sw':
            return 10/8
        if txt=='sww' or txt=='wsw':
            return 11/8
        if txt=='w':
            return 12/8
        if txt=='wnw' or txt=='nww':
            return 13/8
        if txt=='nw':
            return 14/8
        if txt=='nwn' or txt=='nnw':
            return 15/8
        return np.nan
        
    def birthday2day(x):
        days = x.split("/")
        return 30 * int(days[0]) + int(days[1]) + 365 * int(days[2])
    
    def height2inch(x):
        height = x.split("-")
        return 12 * int(height[0]) + int(height[1])    
    
    def str_to_float(txt):
        try:
            return float(txt)
        except:
            return -1
    df['PossessionTeam'] = df['PossessionTeam'].map(map_abbr)
    df['HomeTeamAbbr'] = df['HomeTeamAbbr'].map(map_abbr)
    df['VisitorTeamAbbr'] = df['VisitorTeamAbbr'].map(map_abbr)
    
    df['ToLeft'] = df.PlayDirection == "left"
    # Match the NFLId to that play's rusher's ID
    df['IsRusher'] = df.NflId == df.NflIdRusher 
    
    # New feature to show Dir in radians
    df['Dir_rad'] = np.mod(90 - df.Dir, 360) * math.pi/180.0
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    
    # IsOnOffense
    df['IsOnOffense'] = df.Team == df.TeamOnOffense 
    
    df['YardLine_std'] = 100 - df.YardLine
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
              'YardLine_std'
             ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
              'YardLine']
    df['X_std'] = df.X
    df.loc[df.ToLeft, 'X_std'] = 120 - df.loc[df.ToLeft, 'X'] 
    df['Y_std'] = df.Y
    df.loc[df.ToLeft, 'Y_std'] = 160/3 - df.loc[df.ToLeft, 'Y'] 
    df['Orientation_std'] = -90 + df.Orientation
    df['Dir_std'] = df['Dir_rad']
    df.loc[df.ToLeft, 'Dir_std'] = np.mod(np.pi + df.loc[df.ToLeft, 'Dir_rad'], 2*np.pi)
    
    # Clean Position feature
    df['Position'] = df['Position'].apply(clean_position)
    
    # IsOffenseAtHome
    df['IsOffenseAtHome'] = True
    df.loc[df.TeamOnOffense == 'away', 'IsOffenseAtHome'] = False
    
    # OffenseTeam & DefenseTeam
    df['OffenseTeam'] = df['VisitorTeamAbbr']
    df.loc[df.TeamOnOffense == 'home', 'OffenseTeam'] = df['HomeTeamAbbr']    
    df['DefenseTeam'] = df['VisitorTeamAbbr']
    df.loc[df.TeamOnOffense == 'away', 'DefenseTeam'] = df['HomeTeamAbbr']
    
    # OffenseScore
    df['OffenseScore'] = df['HomeScoreBeforePlay']
    df.loc[df.TeamOnOffense == 'away', 'OffenseScore'] = df['VisitorScoreBeforePlay']
    
    # DefenseScore
    df['DefenseScore'] = df['VisitorScoreBeforePlay']
    df.loc[df.TeamOnOffense == 'away', 'DefenseScore'] = df['HomeScoreBeforePlay']
    
    # IsOffenseWinning
    df['IsOffenseWinning'] = False
    df.loc[df.OffenseScore > df.DefenseScore, 'IsOffenseWinning'] = True
    
    # OffenseInOwnTerritory
    df['OffenseInOwnTerritory'] = False
    df.loc[df.FieldPosition == df.OffenseTeam, 'OffenseInOwnTerritory'] = True
    
    # OffenseRushingPosition
    play_rushers = df.loc[df.NflIdRusher == df.NflId, ['PlayId', 'Position']]
    play_rushers = play_rushers.rename(columns={'Position': 'OffenseRushingPosition'})
    df = df.merge(play_rushers, how='outer', left_on='PlayId', right_on='PlayId')
    
    # OffenceFormation
    df['OffenseFormation'] = df['OffenseFormation'].apply(clean_offenceformation)
    df['OffenseFormation'] = df['OffenseFormation'].fillna(7)  
    
    
    # NumberOfTEsOnPlay, NumberOfWRsOnPlay, NumberOfBacksOnPlay, ....
    df['GeneralPosition'] = df['Position'].apply(create_generalposition)
    df['NumberOfBacksOnPlay'] = 0
    df['NumberOfOLinemenOnPlay'] = 0
    df['NumberOfWRsOnPlay'] = 0
    df['NumberOfTEsOnPlay'] = 0
    df['NumberOfDBsOnPlay'] = 0
    df['NumberOfDLinemenOnPlay'] = 0 
    df['NumberOfLBsOnPlay'] = 0
    # Pivot to find counts of each general position
    gen_pos_counts = df[['PlayId','GeneralPosition']].pivot_table(index='PlayId', columns='GeneralPosition', 
                                                                  aggfunc=len, fill_value=0)
    
    gen_pos_counts = gen_pos_counts.rename(columns = 
                          {'DB':'NumberOfDBsOnPlay', 'DL':'NumberOfDLinemenOnPlay', 
                           'LB':'NumberOfLBsOnPlay', 'OB':'NumberOfBacksOnPlay',
                           'OL':'NumberOfOLinemenOnPlay', 'TE':'NumberOfTEsOnPlay',
                           'WR':'NumberOfWRsOnPlay'})
    gen_pos_counts = gen_pos_counts.reset_index(drop=False)
    del gen_pos_counts.columns.name
    gen_pos_counts_cols = gen_pos_counts.columns.values.tolist()
    gen_pos_counts = gen_pos_counts.loc[gen_pos_counts.index.repeat(22)].reset_index(drop=True)
    df.update(gen_pos_counts)
    
    
    # DefendersInTheBox
    df['DefendersInTheBox'] = df['DefendersInTheBox'].fillna(df['DefendersInTheBox'].median())
    
    # TimeBetweenSnapHandoff, Month, ...
    df['TimeBetweenSnapHandoff'] = df['TimeHandoff'].apply(utc2sec) - df['TimeSnap'].apply(utc2sec)
    df['MatchDay'] = df['TimeSnap'].apply(timesnap2day)
    df['DayOfYear'] = pd.to_datetime(df['TimeSnap'].apply(timesnap2date)).dt.dayofyear
    df['DayOfWeek'] = pd.to_datetime(df['TimeSnap'].apply(timesnap2date)).dt.dayofweek
    df['MonthOfYear'] = df['TimeSnap'].apply(lambda x : int(x[5:7]))
    df['Morning'] = df['TimeSnap'].apply(lambda x : 1 if (int(x[11:13]) >=0 and int(x[11:13]) <12) else 0)
    df['Afternoon'] = df['TimeSnap'].apply(lambda x : 1 if (int(x[11:13]) <18 and int(x[11:13]) >=12) else 0)
    df['Evening'] = df['TimeSnap'].apply(lambda x : 1 if (int(x[11:13]) >= 18 and int(x[11:13]) < 24) else 0)
    
    # QuarterGameSecs, TotalGameSecsPlayed, HalfGameSecs
    df['QuarterGameSecs'] = df['GameClock'].apply(gameclock2secs)
    df['TotalGameSecsPlayed'] = (900 - df['QuarterGameSecs']) + ((df['Quarter'] - 1) * 900)
    df['HalfGameSecsLeft'] = df['QuarterGameSecs']
    df.loc[(df['Quarter'].isin([1,3])), 'HalfGameSecsLeft'] = (900 + df['QuarterGameSecs'])
    
    # IsInEngland
    df['IsInEngland'] = df["Location"].str.lower().map(lambda x: True if "london" in x else False)
    
    # StadiumType
    # from https://www.kaggle.com/code1110/optimizing-lightgbm-hyperparameters
    df['StadiumType'] = df['StadiumType'].apply(group_stadium_types)
    
    # Turf
    # from https://www.kaggle.com/c/nfl-big-data-bowl-2020/discussion/112681#latest-649087
    df['Turf'] = df['Turf'].map(Turf)
    df['Turf'] = df['Turf'].map({"Natural": 0,"Artificial": 1})
    
    # GameWeather
    # https://www.kaggle.com/code1110/optimizing-lightgbm-hyperparameters
    #df['GameWeather'] = df['GameWeather'].apply(group_game_weather)
    df['GameWeather'] = df['GameWeather'].str.lower()
    indoor = "indoor"
    df['GameWeather'] = df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    df['GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    df['GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    df['GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    df['GameWeather'] = df['GameWeather'].apply(map_weather)
    
        
    # Temperature 
    df['Temperature'] = df['Temperature'].fillna(df['Temperature'].median())
    
    # Humidity
    df['Humidity'] = df['Humidity'].fillna(df['Humidity'].median())
    
    # WindSpeed
    #df['WindSpeed'] = df['WindSpeed'].apply(clean_wind_speed)
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: x.lower().replace('mph', '').strip() if not pd.isna(x) else x)
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
    df['WindSpeed'] = df['WindSpeed'].apply(str_to_float)
    
    # WindDirection
    df['WindDirection'] = df['WindDirection'].apply(clean_WindDirection)    
    df['WindDirection'] = df['WindDirection'].apply(transform_WindDirection)
    
    # Team
    df['Team'] = df['Team'].map({"home": 0, "away": 1})
    
    # Dir
    df["Dir"] = np.mod(90 - df["Dir"].values, 360)
    
    # PlayerBirthDate
    df['PlayerBirthDate'] = df['PlayerBirthDate'].apply(birthday2day)
    
    # PlayerAge
    df['PlayerAge'] = df['MatchDay'] - df['PlayerBirthDate']
    
    # PlayDirection
    df['PlayDirection'] = df['PlayDirection'].apply(lambda x: x.strip() == 'right')
    
    # PlayerWeight
    df['PlayerHeight'] = df['PlayerHeight'].apply(height2inch)
    
    # PlayerBMI
    df['PlayerBMI'] = df['PlayerWeight'] / df['PlayerHeight']
    
    # DefendersInTheBox_vs_Distance
    # from https://www.kaggle.com/ryches/model-free-benchmark
    df['DefendersInTheBox_vs_Distance'] = df['DefendersInTheBox'] / df['Distance']
    
    # Start
    # from https://www.kaggle.com/sryo188558/cox-proportional-hazard-model
    df["Start"] = df["YardLine"]
    df.loc[(df["OffenseInOwnTerritory"] == 1) & (df["PlayDirection"] == 1), "Start"] = df.loc[(df["OffenseInOwnTerritory"] == 1) & (df["PlayDirection"] == 1), 
                                                                                       "YardLine"] + 10
    df.loc[(df["OffenseInOwnTerritory"] == 1) & (df["PlayDirection"] == -1), "Start"] = 120 - df.loc[(df["OffenseInOwnTerritory"] == 1) & (df["PlayDirection"] == -1), 
                                                                                       "YardLine"] - 10
    df.loc[(df["OffenseInOwnTerritory"] == 0) & (df["PlayDirection"] == 1), "Start"] = 120 - df.loc[(df["OffenseInOwnTerritory"] == 0) & (df["PlayDirection"] == 1), 
                                                                                       "YardLine"] - 10
    df.loc[(df["OffenseInOwnTerritory"] == 0) & (df["PlayDirection"] == -1), "Start"] = df.loc[(df["OffenseInOwnTerritory"] == 0) & (df["PlayDirection"] == -1), 
                                                                                       "YardLine"] + 10
    # Orientation 
    df['Orientation'] = 2 * np.pi * (90 - df['Orientation']) / 360
    
    # locX
    #df['locX'] = (df['X'].values - df['Start'].values) * df['PlayDirection'].values
    #
    ## locY
    #df['locY'] = df['Y'].values - 53.3 / 2
    #
    ## velX
    #df['velX'] = df['S'].values * np.cos(df['Orientation'].values) * df['PlayDirection'].values
    #
    ## velY
    #df['velY'] = df['S'].values * np.sin(df['Orientation'].values)
    #
    ## accX
    #df['accX'] = df['A'].values * np.cos(df['Orientation'].values) * df['PlayDirection'].values
    #
    ## accY
    #df['accY'] = df['A'].values * np.sin(df['Orientation'].values)
    
    # HomeField
    df['HomeField'] = df['FieldPosition'] == df['HomeTeamAbbr']
    
    # YardsLeft
    df['YardsLeft'] = df.apply(lambda row: 100-row['YardLine'] if row['HomeField'] else row['YardLine'], axis=1)
    df['YardsLeft'] = df.apply(lambda row: row['YardsLeft'] if row['PlayDirection'] else 100-row['YardsLeft'], axis=1)    
    

    df = df.drop(['NflId', 'NflIdRusher','TimeHandoff','TimeSnap','PlayerBirthDate',
                 'Dir_rad','IsOnOffense','X_std','Y_std','Orientation_std','Dir_std'], axis=1)
    
    df = df.select_dtypes(exclude=['object'])
    
    #df.drop(df.index[(df['YardsLeft']<df['Yards']) | (df['YardsLeft']-100>df['Yards'])], inplace=True)
    
    df = df.sort_values(by=['PlayId', 'Team', 'IsRusher', 'JerseyNumber']).reset_index()
    df.drop(['GameId', 'PlayId', 'index', 'IsRusher', 'Team'], axis=1, inplace=True)
    
    df.fillna(-999, inplace=True)

    return(df)

In [ ]:
train = preprocess(df)
train_cols = train.columns

print(train.shape)
train.head()

In [ ]:
# Find cols that are not uniform in each play, append to players_col
players_col = []
for col in train.columns:
    if train[col][:22].std()!=0:
        players_col.append(col)

X_train = train.drop('Yards', axis=1)
X_train = np.array(X_train[players_col]).reshape(-1, len(players_col)*22)

play_col = train.drop(players_col+['Yards'], axis=1).columns
X_play_col = np.zeros(shape=(X_train.shape[0], len(play_col)))
for i, col in enumerate(play_col):
    X_play_col[:, i] = train[col][::22]
 
X_train = np.concatenate([X_train, X_play_col], axis=1)
y_train = np.zeros(shape=(X_train.shape[0], 199))
for i,yard in enumerate(train['Yards'][::22]):
    y_train[i, yard+99:] = np.ones(shape=(1, 100-yard))
    
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:
print("players_col length & type: ", len(players_col), type(players_col))
print("play_col shape & type: ", play_col.shape, type(play_col))
print("X_play_col shape & type: ", X_play_col.shape, type(X_play_col))
print("X_train shape & type: ", X_train.shape, type(X_train))
print("y_train shape & type: ", y_train.shape, type(y_train))

## Modelling

In [ ]:
batch_size=64

class RAdam(keras.optimizers.Optimizer):
    """RAdam optimizer.
    # Arguments
        learning_rate: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        weight_decay: float >= 0. Weight decay for each param.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
        total_steps: int >= 0. Total number of training steps. Enable warmup by setting a positive value.
        warmup_proportion: 0 < warmup_proportion < 1. The proportion of increasing steps.
        min_lr: float >= 0. Minimum learning rate after warmup.
    # References
        - [Adam - A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)
        - [On The Variance Of The Adaptive Learning Rate And Beyond](https://arxiv.org/pdf/1908.03265v1.pdf)
    """

    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., weight_decay=0., amsgrad=False,
                 total_steps=0, warmup_proportion=0.1, min_lr=0., **kwargs):
        learning_rate = kwargs.pop('lr', learning_rate)
        super(RAdam, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.learning_rate = K.variable(learning_rate, name='learning_rate')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.weight_decay = K.variable(weight_decay, name='weight_decay')
            self.total_steps = K.variable(total_steps, name='total_steps')
            self.warmup_proportion = K.variable(warmup_proportion, name='warmup_proportion')
            self.min_lr = K.variable(min_lr, name='min_lr')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.initial_weight_decay = weight_decay
        self.initial_total_steps = total_steps
        self.amsgrad = amsgrad

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr

        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations, K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1

        if self.initial_total_steps > 0:
            warmup_steps = self.total_steps * self.warmup_proportion
            decay_steps = K.maximum(self.total_steps - warmup_steps, 1)
            decay_rate = (self.min_lr - lr) / decay_steps
            lr = K.switch(
                t <= warmup_steps,
                lr * (t / warmup_steps),
                lr + decay_rate * K.minimum(t - warmup_steps, decay_steps),
            )

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='m_' + str(i)) for (i, p) in enumerate(params)]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='v_' + str(i)) for (i, p) in enumerate(params)]

        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='vhat_' + str(i)) for (i, p) in enumerate(params)]
        else:
            vhats = [K.zeros(1, name='vhat_' + str(i)) for i in range(len(params))]

        self.weights = [self.iterations] + ms + vs + vhats

        beta_1_t = K.pow(self.beta_1, t)
        beta_2_t = K.pow(self.beta_2, t)

        sma_inf = 2.0 / (1.0 - self.beta_2) - 1.0
        sma_t = sma_inf - 2.0 * t * beta_2_t / (1.0 - beta_2_t)

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)

            m_corr_t = m_t / (1.0 - beta_1_t)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                v_corr_t = K.sqrt(vhat_t / (1.0 - beta_2_t))
                self.updates.append(K.update(vhat, vhat_t))
            else:
                v_corr_t = K.sqrt(v_t / (1.0 - beta_2_t))

            r_t = K.sqrt((sma_t - 4.0) / (sma_inf - 4.0) *
                         (sma_t - 2.0) / (sma_inf - 2.0) *
                         sma_inf / sma_t)

            p_t = K.switch(sma_t >= 5, r_t * m_corr_t / (v_corr_t + self.epsilon), m_corr_t)

            if self.initial_weight_decay > 0:
                p_t += self.weight_decay * p

            p_t = p - lr * p_t

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    @property
    def lr(self):
        return self.learning_rate

    @lr.setter
    def lr(self, learning_rate):
        self.learning_rate = learning_rate

    def get_config(self):
        config = {
            'learning_rate': float(K.get_value(self.learning_rate)),
            'beta_1': float(K.get_value(self.beta_1)),
            'beta_2': float(K.get_value(self.beta_2)),
            'decay': float(K.get_value(self.decay)),
            'weight_decay': float(K.get_value(self.weight_decay)),
            'epsilon': self.epsilon,
            'amsgrad': self.amsgrad,
            'total_steps': float(K.get_value(self.total_steps)),
            'warmup_proportion': float(K.get_value(self.warmup_proportion)),
            'min_lr': float(K.get_value(self.min_lr)),
        }
        base_config = super(RAdam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
#from https://www.kaggle.com/davidcairuz/nfl-neural-network-w-softmax
def crps(y_true, y_pred):
    return K.mean(K.square(y_true - K.cumsum(y_pred, axis=1)), axis=1)

In [ ]:
def get_model():
    x = keras.layers.Input(shape=[X_train.shape[1]])
    fc1 = keras.layers.Dense(units=450, input_shape=[X_train.shape[1]])(x)
    act1 = keras.layers.PReLU()(fc1)
    bn1 = keras.layers.BatchNormalization()(act1)
    dp1 = keras.layers.Dropout(0.55)(bn1)
    gn1 = keras.layers.GaussianNoise(0.15)(dp1)
    concat1 = keras.layers.Concatenate()([x, gn1])
    fc2 = keras.layers.Dense(units=600)(concat1)
    act2 = keras.layers.PReLU()(fc2)
    bn2 = keras.layers.BatchNormalization()(act2)
    dp2 = keras.layers.Dropout(0.55)(bn2)
    gn2 = keras.layers.GaussianNoise(0.15)(dp2)
    concat2 = keras.layers.Concatenate()([concat1, gn2])
    fc3 = keras.layers.Dense(units=400)(concat2)
    act3 = keras.layers.PReLU()(fc3)
    bn3 = keras.layers.BatchNormalization()(act3)
    dp3 = keras.layers.Dropout(0.55)(bn3)
    gn3 = keras.layers.GaussianNoise(0.15)(dp3)
    concat3 = keras.layers.Concatenate([concat2, gn3])
    output = keras.layers.Dense(units=199, activation='softmax')(concat2)
    model = keras.models.Model(inputs=[x], outputs=[output])
    return model


def train_model(X_train, y_train, X_val, y_val):
    model = get_model()
    model.compile(optimizer=RAdam(warmup_proportion=0.1, min_lr=1e-7), loss=crps)
    er = EarlyStopping(patience=20, min_delta=1e-4, restore_best_weights=True, monitor='val_loss')
    model.fit(X_train, y_train, epochs=200, callbacks=[er], validation_data=[X_val, y_val], batch_size=batch_size)
    return model

In [ ]:
from sklearn.model_selection import RepeatedKFold

rkf = RepeatedKFold(n_splits=5, n_repeats=5)

models = []

for tr_idx, vl_idx in rkf.split(X_train, y_train):
    
    x_tr, y_tr = X_train[tr_idx], y_train[tr_idx]
    x_vl, y_vl = X_train[vl_idx], y_train[vl_idx]
    
    model = train_model(x_tr, y_tr, x_vl, y_vl)
    models.append(model)

In [ ]:
print(np.mean(score))

## Submission

In [ ]:
def make_pred(df, sample, env, models):
    #print('#### BEGINNING #### ')
    #print('Raw df shape: ', df.shape)
    #print('Running df preprocessing...')
    df = preprocess(df)
    #df_cols = df.columns

    #print('df preprocessing complete')
    #print('df shape: ', df.shape)
    #print(' ')
    
    #print('Difference between train and df cols')
    #col_diff = list(set(train_cols) - set(df_cols))
    #print(col_diff)
    #print(' ')
    
    #print('Reshaping X and removing players_col...')
    X = np.array(df[players_col]).reshape(-1, len(players_col)*22)
    #print('Reshape complete')
    #print('X shape: ', X.shape)
    #print(' ')

    
    #print('Defining play_col...')
    play_col = df.drop(players_col, axis=1).columns
    #print('play_col shape: ', play_col.shape)
    #print(' ')
    
    
    #print('Defining X_play_col...')
    X_play_col = np.zeros(shape=(X.shape[0], len(play_col)))
    #print('X_play_col shape(1): ', X_play_col.shape)
    #print(' ')

    
    #print('Selecting X_play_col data...')
    for i, col in enumerate(play_col):
        X_play_col[:, i] = df[col][::22]
    #print('X_play_col shape(2): ', X_play_col.shape)
    #print(' ')
    
    
    #print('Concatenating X with X_play_col...')
    X = np.concatenate([X, X_play_col], axis=1)
    #print('X concatenation complete')
    #print('X shape: ', X.shape)
    #print(' ')

    
    #print('Scaling X...')
    X = scaler.transform(X)
    #print('Scaling X complete')
    #print('X shape: ', X.shape)
    #print(' ')

    
    #print('Predicting y_pred...')    
    y_pred = np.mean([np.cumsum(model.predict(X), axis=1) for model in models], axis=0)
    #print('Prediction complete')
    #print('y_pred shape: ', y_pred.shape)
    #print(' ')

    
    #print('Defining YardsLeft...')
    yardsleft = np.array(df['YardsLeft'][::22])
    #print('YardsLeft shape: ', yardsleft.shape)
    #print(' ')

    
    #print('Looping y_pred for final sub...')
    for i in range(len(yardsleft)):
        y_pred[i, :yardsleft[i]-1] = 0
        y_pred[i, yardsleft[i]+100:] = 1
    #print('y_pred loop complete')
    #print('y_pred shape: ', y_pred.shape)
    #print(' ')

    
    #print('Finalising submission')
    env.predict(pd.DataFrame(data=y_pred.clip(0,1),columns=sample.columns))
    #print('Submission complete. Moving to next iteration.')
    #print('#### END ####')
    #print(' ')
    #print(' ')
    #print(y_pred)
    return y_pred

In [ ]:
from kaggle.competitions import nflrush
env = nflrush.make_env()

for test, sample in tqdm.tqdm(env.iter_test()):
    make_pred(test, sample, env, models)
env.write_submission_file()

In [ ]:
## from https://www.kaggle.com/newbielch/lgbm-regression-view
#def get_cdf_df(yards_array):
#    pdf, edges = np.histogram(yards_array, bins=199,
#                 range=(-99,100), density=True)
#    cdf = pdf.cumsum().clip(0, 1)
#    cdf_df = pd.DataFrame(data=cdf.reshape(-1, 1).T, 
#                            columns=['Yards'+str(i) for i in range(-99,100)])
#    return cdf_df
#cdf = get_cdf_df(y_train).values.reshape(-1,)
##dist_to_end_train = X_train.apply(lambda x:(100 - x.loc['YardLine']) if x.loc["OffenseInOwnTerritory"]==1 else x.loc['YardLine'],axis=1)

In [ ]:
#def get_score(y_pred,cdf,w,dist_to_end):
#    y_pred = int(y_pred)
##     y_pred = y_pred.astype(int)
#    if y_pred ==w:
#        y_pred_array = cdf.copy()
#    elif y_pred - w >0:
#        y_pred_array = np.zeros(199)
#        y_pred_array[(y_pred-w):] = cdf[:(-(y_pred-w))].copy()
#    elif w - y_pred >0:
#        y_pred_array = np.ones(199)
#        y_pred_array[:(y_pred-w)] = cdf[(w-y_pred):].copy()
#    y_pred_array[-1]=1
#    y_pred_array[(dist_to_end+99):]=1
#    return y_pred_array    
#
#def get_score_pingyi1(y_pred,y_true,cdf,w,dist_to_end):
#    y_pred = int(y_pred)
#    if y_pred ==w:
#        y_pred_array = cdf.copy()
#    elif y_pred - w >0:
#        y_pred_array = np.zeros(199)
#        y_pred_array[(y_pred-w):] = cdf[:(-(y_pred-w))].copy()
#    elif w - y_pred >0:
#        y_pred_array = np.ones(199)
#        y_pred_array[:(y_pred-w)] = cdf[(w-y_pred):].copy()
#    y_pred_array[-1]=1
#    y_pred_array[(dist_to_end+99):]=1
#    y_true_array = np.zeros(199)
#    y_true_array[(y_true+99):]=1
#    return np.mean((y_pred_array - y_true_array)**2)
#
#
#def CRPS_pingyi1(y_preds,y_trues,w,cdf,dist_to_ends):
#    if len(y_preds) != len(y_trues):
#        print('length does not match')
#        return None
#    n = len(y_preds)
#    tmp = []
#    for a,b,c in zip(y_preds, y_trues, dist_to_ends):
#        tmp.append(get_score_pingyi1(a,b,cdf,w,c))
#    return np.mean(tmp)

In [ ]:
## Initial LGB parameters are ...
#lgbParams = {
#    'objective': 'regression',
#    'metric': 'mae',
#    'verbosity': -1,
#    'boosting_type': 'gbdt',
#    "num_iterations": 1000, 
#    "learning_rate": 0.05,
#    "lambda_l1": 9,
#    "lambda_l2": 0.9,
#    "num_leaves": 42,
#    "feature_fraction": 0.4,
#    "bagging_fraction": 0.45,
#    "bagging_freq": 7,
#    "min_child_samples": 74,
#    "random_state": 14
#}

In [ ]:
### Visualize feature importance
#
## make a LightGBM dataset
#trainX, testX, trainY, testY = train_test_split(X_train, y_train, test_size=0.33, random_state=14)
#d_train = lgb.Dataset(trainX, trainY)
#d_eval = lgb.Dataset(testX, testY, reference=d_train)
#
## model training
#LGBmodel = lgb.train(lgbParams, d_train, valid_sets=d_eval, verbose_eval=1000)
## LGBmodel = lgb.train(lgbParams, d_train, valid_sets=d_eval, early_stopping_rounds=500, verbose_eval=1000)
#
## feature importance
#importance = LGBmodel.feature_importance(importance_type="gain")
#ranking = np.argsort(-importance)
#fig, ax = plt.subplots(figsize=(20, 20))
#sns.barplot(x=importance[ranking], y=X_train.columns.values[ranking], orient='h')
#ax.set_xlabel("feature importance")
#plt.tight_layout()

In [ ]:
#features = X_train.columns.values[ranking][:30]
#print(features)
#X_train = X_train[features]

In [ ]:
## FYI: Objective functions can take additional arguments
## (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
#def objective(trial):
#  
#    # make a LightGBM dataset
#    trainX, testX, trainY, testY = train_test_split(X_train, y_train, test_size=0.33, random_state=14)
#    d_train = lgb.Dataset(trainX, trainY)
#
#    param = {
#        'objective': 'regression',
#        'metric': 'mae',
#        'verbosity': -1,
#        'boosting_type': 'gbdt',
#        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
#        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
#        'num_leaves': trial.suggest_int('num_leaves', 40, 256),
#        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
#        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
#        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
#        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#    }
#
#    gbm = lgb.train(param, d_train)
#    preds = gbm.predict(testX)
#    mae = mean_absolute_error(testY, preds)
#    return mae

In [ ]:
#study = optuna.create_study(direction='minimize')
#study.optimize(objective, n_trials=100)
#
#print('Number of finished trials: {}'.format(len(study.trials)))
#
#print('Best trial:')
#trial = study.best_trial
#
#print('  Value: {}'.format(trial.value))
#
#print('  Params: ')
#for key, value in trial.params.items():
#    print('    {}: {}'.format(key, value))

In [ ]:
#lgbParams = trial.params
#lgbParams['objective'] = 'regression'
#lgbParams['metric'] = 'mae'
#lgbParams['verbosity'] = -1
#lgbParams['boosting_type'] = 'gbdt'
#lgbParams["learning_rate"] = 0.01
#lgbParams["num_iterations"] = 5000
#lgbParams["random_state"] = 14
#print(lgbParams)

In [ ]:
#lgbParams = {'lambda_l1': 9.830445878717612, 
#             'lambda_l2': 9.889595129567367, 
#             'num_leaves': 41, 
#             'feature_fraction': 0.4152661948258711,
#             'bagging_fraction': 0.9170414824158851, 
#             'bagging_freq': 7, 
#             'min_child_samples': 44, 
#             'objective': 'regression', 
#             'metric': 'mae', 
#             'verbosity': -1, 
#             'boosting_type': 'gbdt', 
#             'learning_rate': 0.01, 
#             'num_iterations': 5000, 
#             'random_state': 14}

In [ ]:
#n_splits = 5
#seed = 14
#kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
#y_valid = np.zeros(X_train.shape[0])
#models = []
#
#for train_idx, valid_idx in kf.split(X_train, y_train):
#    trainX, trainY = X_train.iloc[train_idx, :], y_train[train_idx]
#    validX, validY = X_train.iloc[valid_idx, :], y_train[valid_idx]
#    
#    d_train = lgb.Dataset(trainX, trainY)
#    d_eval = lgb.Dataset(validX, validY, reference=d_train)
#    
#    LGBmodel = lgb.train(lgbParams, d_train, valid_sets=d_eval, 
#                         early_stopping_rounds=500, 
#                         learning_rates = lambda iter: 0.01 * (0.99 ** iter),
#                         verbose_eval = 1000)
#    y_valid[valid_idx] += LGBmodel.predict(validX, num_iteration=LGBmodel.best_iteration)
#    models.append(LGBmodel)
#gc.collect()

In [ ]:
#cprs = CRPS_pingyi1(y_valid, y_train.astype(int), 4, cdf, dist_to_end_train.astype(int))
#print("cprs = {}".format(cprs))

## Submission

In [ ]:
#from kaggle.competitions import nflrush
#env = nflrush.make_env()

In [ ]:
#index = 0
#for (test_df, sample_prediction_df) in tqdm.tqdm(env.iter_test()):
#    test = preprocess(test_df)
#    
#    count=0
#    test_data = np.zeros((1,len(features)))
#    for c in features:
#        try:
#            test_data[0][count] = test[c][index]
#        except:
#            test_data[0][count] = np.nan
#        count+=1
#    
#    y_pred = np.zeros(199)    
#    y_pred_p = np.sum(np.round(scaler.inverse_transform(
#        [model.predict(test_data) for model in models]))) / n_splits
#    
#    y_pred_p += 99
#    for j in range(199):
#        if j>=y_pred_p+10:
#            y_pred[j]=1.0
#        elif j>=y_pred_p-10:
#            y_pred[j]=(j+10-y_pred_p)*0.05
#
#    env.predict(pd.DataFrame(data=[y_pred],columns=sample_prediction_df.columns))
#    index += 22
#env.write_submission_file()